In [4]:
"""This is an ACT-R model of the Iowa Gambling Task using reinforcement learning"""
"""Written by Jim Treyens"""
"""11/13/2019"""

"""The model selects 100 cards from 4 decks as described in Bechara et al. (1994)  """

""" Bechara, A., Damasio, A. R., Damasio, H., & Anderson, S. W. (1994). Insensitivity to future consequences   """ 
""" following damage to human prefrontal cortex. Cognition, 50(1-3), 7-15."""

""" The model produces a trace showing information for each card selected: """
"""  Overall pick #, deck from card was picked, penalty associated with pick, reward (where reward = yield - penalty) """

"""Decide whether to put import statements in Jupyter notebook or in python file"""

import actr
import random as rnd
import numpy as np
import os
import sys
import string

actr.load_act_r_model("ACT-R:IGT;IGT_RL_model.lisp")

decka_penalties = [0,0,150,0,300,0,200,0,250,350,0,350,0,250,200,0,300,150,0,0,0,300,0,350,0,200,250,150,0,0,350,200,250,0,0,0,150,300,0,0]
decka_counter = 0

deckb_penalties = [0,0,0,0,0,0,0,0,1250,0,0,0,0,1250,0,0,0,0,0,0,1250,0,0,0,0,0,0,0,0,0,0,1250,0,0,0,0,0,0,0,0]
deckb_counter = 0

deckc_penalties = [0,0,50,0,50,0,50,0,50,50,0,25,75,0,0,0,25,75,0,50,0,0,0,50,25,50,0,0,75,50,0,0,0,25,25,0,75,0,50,75]
deckc_counter = 0

deckd_penalties = [0,0,0,0,0,0,0,0,0,250,0,0,0,0,0,0,0,0,0,250,0,0,0,0,0,0,0,0,250,0,0,0,0,0,250,0,0,0,0,0]
deckd_counter = 0

current_pick = None
total_picks = 0
reward = 0
trace = []


"""Loads decks to visual buffer""" 
def load_decks():
    global decka_counter
    global deckb_counter
    global deckc_counter
    global deckd_counter
    if decka_counter < 40:
        decka = "yes"
    else:
        decka = "no"    
    if deckb_counter < 40:
        deckb = "yes"
    else:
        deckb = "no"                     
    if deckc_counter < 40:
        deckc = "yes"
    else:
        deckc = "no"        
    if deckc_counter < 40:
        deckd = "yes"
    else:
        deckd = "no"                
    
    deck_chunk = actr.define_chunks(['isa', 'decks', 'deckA', decka, 'deckB', deckb, 'deckC', deckc, 'deckD', deckd])

    actr.set_buffer_chunk("visual", deck_chunk[0])

"""Responds to key press and calculates reward"""
def respond_to_key_press(model,key):
    global current_pick
    current_pick = key
    global total_picks
    global reward
    global decka_counter
    global deckb_counter
    global deckc_counter
    global deckd_counter
    global trace
    
    if current_pick == "a":   
        reward = (100 - decka_penalties[decka_counter])
        decka_counter = decka_counter + 1
        total_picks = total_picks+1
        actr.schedule_event_now("load_reward")
        trace.append(total_picks)
        trace.append(current_pick)
        trace.append(decka_penalties[decka_counter-1])
        trace.append(reward)
    
    if current_pick == "b":
        reward = (100 - deckb_penalties[deckb_counter])
        deckb_counter = deckb_counter + 1
        total_picks = total_picks+1
        actr.schedule_event_now("load_reward")
        trace.append(total_picks)
        trace.append(current_pick)
        trace.append(deckb_penalties[deckb_counter-1])
        trace.append(reward)        
        
    
    if current_pick == "c":
        reward = (50 - deckc_penalties[deckc_counter])
        deckc_counter = deckc_counter + 1
        total_picks = total_picks+1
        actr.schedule_event_now("load_reward")
        trace.append(total_picks)
        trace.append(current_pick)
        trace.append(deckc_penalties[deckc_counter-1]) 
        trace.append(reward)
        
    if current_pick == "d":
        reward = (50 - deckd_penalties[deckd_counter])
        deckd_counter = deckd_counter + 1
        total_picks = total_picks+1
        actr.schedule_event_now("load_reward")
        trace.append(total_picks)
        trace.append(current_pick)        
        trace.append(deckd_penalties[deckd_counter-1])
        trace.append(reward)
        
"""Loads reward to visual buffer"""
def load_reward():
    global reward
    reward_chunk = actr.define_chunks(['isa', 'reward-amount', 'amount', reward])
    actr.set_buffer_chunk("visual", reward_chunk[0])
    
"""THE NEXT TWO LINES ARE CRITICAL FOR INTERFACING WITH THE ACT-R/LISP CODE; THEY ARE REQUIRED FOR PYTHON TO DETECT KEY PRESSES"""    
win = actr.open_exp_window("Test", visible=False)
actr.install_device(win)

"""THE FOLLOWING LINES SET UP THE INTERFACE TO ACT-R/LISP CODE"""
actr.add_command("get-pick", respond_to_key_press, "IGT key press response monitor")
actr.monitor_command("output-key","get-pick")
actr.add_command("load_reward", load_reward, "Loads reward amount chunk to visual buffer")
actr.add_command("load_decks", load_decks, "Loads decks chunk to visual buffer")

event_time = 1
event_step = 1
total_picks_for_while_loop = 0

while total_picks_for_while_loop < 100:
    actr.schedule_event(event_time, "load_decks")
    event_time += event_step
    total_picks_for_while_loop = total_picks_for_while_loop + 1
    
actr.run(150)


actr.remove_command_monitor("output-key", "get-pick")
actr.remove_command("get-pick")
actr.remove_command("load_reward")
actr.remove_command("load_decks")

# print("Pick %s" % (current_pick))
print("trace picks: ",trace)





#|Warning: Creating chunk YES with no slots |#
     0.000   PROCEDURAL             CONFLICT-RESOLUTION
     1.000   PROCEDURAL             CONFLICT-RESOLUTION
     1.050   PROCEDURAL             PRODUCTION-FIRED SELECT-D
     1.050   PROCEDURAL             CLEAR-BUFFER VISUAL
     1.050   PROCEDURAL             CLEAR-BUFFER MANUAL
     1.050   MOTOR                  PRESS-KEY KEY d
     1.050   PROCEDURAL             CONFLICT-RESOLUTION
     1.200   PROCEDURAL             CONFLICT-RESOLUTION
     1.250   PROCEDURAL             CONFLICT-RESOLUTION
     1.260   PROCEDURAL             CONFLICT-RESOLUTION
     1.350   PROCEDURAL             CONFLICT-RESOLUTION
     1.400   PROCEDURAL             PRODUCTION-FIRED PROCESS_REWARD+50
     1.400   PROCEDURAL             CLEAR-BUFFER VISUAL
     1.400   PROCEDURAL             CLEAR-BUFFER MANUAL
     1.400   MOTOR                  PRESS-KEY KEY SPACE
     1.400   UTILITY                PROPAGATE-REWARD 50
     1.400   PROCEDURAL             CONF

     9.050   PROCEDURAL             CONFLICT-RESOLUTION
     9.300   PROCEDURAL             CONFLICT-RESOLUTION
     9.350   PROCEDURAL             CONFLICT-RESOLUTION
     9.450   PROCEDURAL             CONFLICT-RESOLUTION
     9.600   PROCEDURAL             CONFLICT-RESOLUTION
     9.650   PROCEDURAL             PRODUCTION-FIRED PROCESS_REWARD+100
     9.650   PROCEDURAL             CLEAR-BUFFER VISUAL
     9.650   PROCEDURAL             CLEAR-BUFFER MANUAL
     9.650   MOTOR                  PRESS-KEY KEY SPACE
     9.650   UTILITY                PROPAGATE-REWARD 100
     9.650   PROCEDURAL             CONFLICT-RESOLUTION
     9.800   PROCEDURAL             CONFLICT-RESOLUTION
     9.850   PROCEDURAL             CONFLICT-RESOLUTION
     9.860   PROCEDURAL             CONFLICT-RESOLUTION
     9.950   PROCEDURAL             CONFLICT-RESOLUTION
    10.000   PROCEDURAL             CONFLICT-RESOLUTION
    10.050   PROCEDURAL             PRODUCTION-FIRED SELECT-B
    10.050   PROCEDURAL  

    16.650   PROCEDURAL             CONFLICT-RESOLUTION
    16.800   PROCEDURAL             CONFLICT-RESOLUTION
    16.850   PROCEDURAL             CONFLICT-RESOLUTION
    16.860   PROCEDURAL             CONFLICT-RESOLUTION
    16.950   PROCEDURAL             CONFLICT-RESOLUTION
    17.000   PROCEDURAL             CONFLICT-RESOLUTION
    17.050   PROCEDURAL             PRODUCTION-FIRED SELECT-D
    17.050   PROCEDURAL             CLEAR-BUFFER VISUAL
    17.050   PROCEDURAL             CLEAR-BUFFER MANUAL
    17.050   MOTOR                  PRESS-KEY KEY d
    17.050   PROCEDURAL             CONFLICT-RESOLUTION
    17.100   PROCEDURAL             CONFLICT-RESOLUTION
    17.150   PROCEDURAL             CONFLICT-RESOLUTION
    17.160   PROCEDURAL             CONFLICT-RESOLUTION
    17.250   PROCEDURAL             CONFLICT-RESOLUTION
    17.300   PROCEDURAL             PRODUCTION-FIRED PROCESS_REWARD+50
    17.300   PROCEDURAL             CLEAR-BUFFER VISUAL
    17.300   PROCEDURAL        

    24.350   PROCEDURAL             CONFLICT-RESOLUTION
    24.400   PROCEDURAL             CONFLICT-RESOLUTION
    24.410   PROCEDURAL             CONFLICT-RESOLUTION
    24.500   PROCEDURAL             CONFLICT-RESOLUTION
    25.000   PROCEDURAL             CONFLICT-RESOLUTION
    25.050   PROCEDURAL             PRODUCTION-FIRED SELECT-C
    25.050   PROCEDURAL             CLEAR-BUFFER VISUAL
    25.050   PROCEDURAL             CLEAR-BUFFER MANUAL
    25.050   MOTOR                  PRESS-KEY KEY c
    25.050   PROCEDURAL             CONFLICT-RESOLUTION
    25.300   PROCEDURAL             CONFLICT-RESOLUTION
    25.350   PROCEDURAL             CONFLICT-RESOLUTION
    25.450   PROCEDURAL             CONFLICT-RESOLUTION
    25.600   PROCEDURAL             CONFLICT-RESOLUTION
    25.650   PROCEDURAL             PRODUCTION-FIRED PROCESS_REWARD+50
    25.650   PROCEDURAL             CLEAR-BUFFER VISUAL
    25.650   PROCEDURAL             CLEAR-BUFFER MANUAL
    25.650   MOTOR             

    33.410   PROCEDURAL             CONFLICT-RESOLUTION
    33.500   PROCEDURAL             CONFLICT-RESOLUTION
    34.000   PROCEDURAL             CONFLICT-RESOLUTION
    34.050   PROCEDURAL             PRODUCTION-FIRED SELECT-C
    34.050   PROCEDURAL             CLEAR-BUFFER VISUAL
    34.050   PROCEDURAL             CLEAR-BUFFER MANUAL
    34.050   MOTOR                  PRESS-KEY KEY c
    34.050   PROCEDURAL             CONFLICT-RESOLUTION
    34.300   PROCEDURAL             CONFLICT-RESOLUTION
    34.350   PROCEDURAL             CONFLICT-RESOLUTION
    34.450   PROCEDURAL             CONFLICT-RESOLUTION
    34.600   PROCEDURAL             CONFLICT-RESOLUTION
    34.650   PROCEDURAL             PRODUCTION-FIRED PROCESS_REWARD-25
    34.650   PROCEDURAL             CLEAR-BUFFER VISUAL
    34.650   PROCEDURAL             CLEAR-BUFFER MANUAL
    34.650   MOTOR                  PRESS-KEY KEY SPACE
    34.650   UTILITY                PROPAGATE-REWARD -25
    34.650   PROCEDURAL       

...

    64.050   PROCEDURAL             PRODUCTION-FIRED SELECT-C
    64.050   PROCEDURAL             CLEAR-BUFFER VISUAL
    64.050   PROCEDURAL             CLEAR-BUFFER MANUAL
    64.050   MOTOR                  PRESS-KEY KEY c
    64.050   PROCEDURAL             CONFLICT-RESOLUTION
    64.300   PROCEDURAL             CONFLICT-RESOLUTION
    64.350   PROCEDURAL             CONFLICT-RESOLUTION
    64.450   PROCEDURAL             CONFLICT-RESOLUTION
    64.600   PROCEDURAL             CONFLICT-RESOLUTION
    64.650   PROCEDURAL             PRODUCTION-FIRED PROCESS_REWARD+50
    64.650   PROCEDURAL             CLEAR-BUFFER VISUAL
    64.650   PROCEDURAL             CLEAR-BUFFER MANUAL
    64.650   MOTOR                  PRESS-KEY KEY SPACE
    64.650   UTILITY                PROPAGATE-REWARD 50
    64.650   PROCEDURAL             CONFLICT-RESOLUTION
    64.800   PROCEDURAL             CONFLICT-RESOLUTION
    64.850   PROCEDURAL             CONFLICT-RESOLUTION
    64.860   PROCEDURAL        

    72.050   PROCEDURAL             PRODUCTION-FIRED SELECT-D
    72.050   PROCEDURAL             CLEAR-BUFFER VISUAL
    72.050   PROCEDURAL             CLEAR-BUFFER MANUAL
    72.050   MOTOR                  PRESS-KEY KEY d
    72.050   PROCEDURAL             CONFLICT-RESOLUTION
    72.100   PROCEDURAL             CONFLICT-RESOLUTION
    72.150   PROCEDURAL             CONFLICT-RESOLUTION
    72.160   PROCEDURAL             CONFLICT-RESOLUTION
    72.250   PROCEDURAL             CONFLICT-RESOLUTION
    72.300   PROCEDURAL             PRODUCTION-FIRED PROCESS_REWARD+50
    72.300   PROCEDURAL             CLEAR-BUFFER VISUAL
    72.300   PROCEDURAL             CLEAR-BUFFER MANUAL
    72.300   MOTOR                  PRESS-KEY KEY SPACE
    72.300   UTILITY                PROPAGATE-REWARD 50
    72.300   PROCEDURAL             CONFLICT-RESOLUTION
    72.350   PROCEDURAL             CONFLICT-RESOLUTION
    72.400   PROCEDURAL             CONFLICT-RESOLUTION
    72.410   PROCEDURAL        

    80.500   PROCEDURAL             CONFLICT-RESOLUTION
    81.000   PROCEDURAL             CONFLICT-RESOLUTION
    81.050   PROCEDURAL             PRODUCTION-FIRED SELECT-D
    81.050   PROCEDURAL             CLEAR-BUFFER VISUAL
    81.050   PROCEDURAL             CLEAR-BUFFER MANUAL
    81.050   MOTOR                  PRESS-KEY KEY d
    81.050   PROCEDURAL             CONFLICT-RESOLUTION
    81.100   PROCEDURAL             CONFLICT-RESOLUTION
    81.150   PROCEDURAL             CONFLICT-RESOLUTION
    81.160   PROCEDURAL             CONFLICT-RESOLUTION
    81.250   PROCEDURAL             CONFLICT-RESOLUTION
    81.300   PROCEDURAL             PRODUCTION-FIRED PROCESS_REWARD+50
    81.300   PROCEDURAL             CLEAR-BUFFER VISUAL
    81.300   PROCEDURAL             CLEAR-BUFFER MANUAL
    81.300   MOTOR                  PRESS-KEY KEY SPACE
    81.300   UTILITY                PROPAGATE-REWARD 50
    81.300   PROCEDURAL             CONFLICT-RESOLUTION
    81.350   PROCEDURAL        

    89.000   PROCEDURAL             CONFLICT-RESOLUTION
    89.050   PROCEDURAL             PRODUCTION-FIRED SELECT-A
    89.050   PROCEDURAL             CLEAR-BUFFER VISUAL
    89.050   PROCEDURAL             CLEAR-BUFFER MANUAL
    89.050   MOTOR                  PRESS-KEY KEY a
    89.050   PROCEDURAL             CONFLICT-RESOLUTION
    89.100   PROCEDURAL             CONFLICT-RESOLUTION
    89.150   PROCEDURAL             CONFLICT-RESOLUTION
    89.160   PROCEDURAL             CONFLICT-RESOLUTION
    89.250   PROCEDURAL             CONFLICT-RESOLUTION
    89.300   PROCEDURAL             PRODUCTION-FIRED PROCESS_REWARD+100
    89.300   PROCEDURAL             CLEAR-BUFFER VISUAL
    89.300   PROCEDURAL             CLEAR-BUFFER MANUAL
    89.300   MOTOR                  PRESS-KEY KEY SPACE
    89.300   UTILITY                PROPAGATE-REWARD 100
    89.300   PROCEDURAL             CONFLICT-RESOLUTION
    89.350   PROCEDURAL             CONFLICT-RESOLUTION
    89.400   PROCEDURAL      

    97.250   PROCEDURAL             CONFLICT-RESOLUTION
    97.300   PROCEDURAL             PRODUCTION-FIRED PROCESS_REWARD+100
    97.300   PROCEDURAL             CLEAR-BUFFER VISUAL
    97.300   PROCEDURAL             CLEAR-BUFFER MANUAL
    97.300   MOTOR                  PRESS-KEY KEY SPACE
    97.300   UTILITY                PROPAGATE-REWARD 100
    97.300   PROCEDURAL             CONFLICT-RESOLUTION
    97.350   PROCEDURAL             CONFLICT-RESOLUTION
    97.400   PROCEDURAL             CONFLICT-RESOLUTION
    97.410   PROCEDURAL             CONFLICT-RESOLUTION
    97.500   PROCEDURAL             CONFLICT-RESOLUTION
    98.000   PROCEDURAL             CONFLICT-RESOLUTION
    98.050   PROCEDURAL             PRODUCTION-FIRED SELECT-A
    98.050   PROCEDURAL             CLEAR-BUFFER VISUAL
    98.050   PROCEDURAL             CLEAR-BUFFER MANUAL
    98.050   MOTOR                  PRESS-KEY KEY a
    98.050   PROCEDURAL             CONFLICT-RESOLUTION
    98.100   PROCEDURAL      